In [1]:
import csv
import os
import xml.etree.ElementTree as ET

## Generating the XML files for all measures combinations 

Here we will use the file 'measures_revised.csv' which contains all similarity measures supported by SML derived from their dedicated [spreadsheet](https://docs.google.com/spreadsheets/u/0/d/1Iru4J7BrgufGmt6gACcHDFnVmmmP3ngf4yXklmG8zHQ/pub?output=html#), each given a unique ID that will be used to label the output scores.


### Set up list to store the measures by type 

In [2]:
measures_source = 'measures_revised.csv' 
# file containing all supported measures as follows:

Non_IC_pairwise_measures = ["structure_pairwise",  
                           "direct_groupwise", 
                            "indirect_groupwise"] # all non-IC based measures, pairwise and groupwise

IC_pairwise_measures = ["ic_measure","ic_pairwise"] # all IC measures and IC-based pairwise measures

std_pairwise_measures = ["structure_pairwise"] # non-IC based pairwise 

std_groupwise_measures = ["direct_groupwise", "set_groupwise"] # standalone groupwise; set_groupwise are all FRAMEWORK measures

IC_groupwise_measures = ["indirect_groupwise"] # all groupwise that require a pairwise measure 


# empty xml files to be edited
DIRECT_GROUPWISE_xml_template = "DIRECT_GROUPWISE_MEASURES_COMBINATIONS.xml" # standalone groupwise (no pairwise or IC)
INDIRECT_GROUPWISE_xml_template = "INDIRECT_GROUPWISE_MEASURES_COMBINATIONS.xml" # indirect groupwise with non-IC pairwise
IC_INDIRECT_GROUPWISE_xml_template = "IC_INDIRECT_GROUPWISE_MEASURES_COMBINATIONS.xml" # indirect groupwise with ICs and IC-based pairwise

### Write IC measures into the relevant XML

In [4]:
# insert all relevant <ic/> tags into the xml file IC_INDIRECT_GROUPWISE_xml_template

with open(measures_source) as pairwise_IC:
    pairwise_IC_csv = csv.reader(pairwise_IC)
    pairwise_IC_headers = next(pairwise_IC_csv)
    for pairwise_IC_item in pairwise_IC_csv:
        if pairwise_IC_item[0] == "ic_measure": # these are all of type 'ic_measure' to include from measures_revised
            IC_tree = ET.parse(IC_INDIRECT_GROUPWISE_xml_template)
            IC_root = IC_tree.getroot()
            IC_element_target = IC_root.find("./sml/ics")
            IC_new_measure = ET.SubElement(IC_element_target, 'ic', 
                                            id = pairwise_IC_item[2], 
                                            flag = pairwise_IC_item[1])
            
            IC_tree.write(IC_INDIRECT_GROUPWISE_xml_template)
            

### Write all pairwise and groupwise measures into the relevant XMLs

In [5]:
with open(measures_source) as pairwise_IC:
    pairwise_IC_csv = csv.reader(pairwise_IC)
    pairwise_IC_headers = next(pairwise_IC_csv)
    for pairwise_IC_item in pairwise_IC_csv:
        
        # generate the pairwise block for IC measures included
        if pairwise_IC_item[0] == "ic_pairwise": # type 'ic_pairwise' will require a specified IC measure as well
            IC_tree = ET.parse(IC_INDIRECT_GROUPWISE_xml_template)
            IC_root = IC_tree.getroot()
            IC_element_target = IC_root.find("./sml/measures/[@type='pairwise']")
            IC_new_measure = ET.SubElement(IC_element_target, 'measure', 
                                            id = pairwise_IC_item[2], 
                                            flag = pairwise_IC_item[1],
                                            ic = "[FULL_LIST_IC]")
                    
            IC_tree.write(IC_INDIRECT_GROUPWISE_xml_template) # writes the IC-based pairwise into the XML with ic=[FULL_LIST_IC]
        
        # generate pairwise block for non-IC based standard pariwise measures
        elif pairwise_IC_item[0] in std_pairwise_measures: 
            IC_tree = ET.parse(INDIRECT_GROUPWISE_xml_template)
            IC_root = IC_tree.getroot()
            
            IC_element_target = IC_root.find("./sml/measures/[@type='pairwise']")
            
            IC_new_measure = ET.SubElement(IC_element_target, 'measure', 
                                            id = pairwise_IC_item[2], 
                                            flag = pairwise_IC_item[1]
                                            )
                    
            IC_tree.write(INDIRECT_GROUPWISE_xml_template) # writes the non-IC based pairwise into the XML
        
        # generate the groupwise block for standalone groupwise measures
        elif pairwise_IC_item[0] in std_groupwise_measures: 
            IC_tree = ET.parse(DIRECT_GROUPWISE_xml_template)
            IC_root = IC_tree.getroot()
            
            IC_element_target = IC_root.find("./sml/measures/[@type='groupwise']")
            
            IC_new_measure = ET.SubElement(IC_element_target, 'measure', 
                                            id = pairwise_IC_item[2], 
                                            flag = pairwise_IC_item[1],
                                            )
                    
            IC_tree.write(DIRECT_GROUPWISE_xml_template) # standalone groupwise measures written into a single XML
        
        # generate the groupwise block for indirect groupwise measures
        elif pairwise_IC_item[0] in IC_groupwise_measures: 
            
            # with [FULL_LIST_PM] of all IC-based pairwise measures                                              
            IC_tree = ET.parse(IC_INDIRECT_GROUPWISE_xml_template)
            IC_root = IC_tree.getroot()
            IC_element_target = IC_root.find("./sml/measures/[@type='groupwise']")
            IC_new_measure = ET.SubElement(IC_element_target, 'measure', 
                                            id = pairwise_IC_item[2], 
                                            flag = pairwise_IC_item[1],
                                            pairwise_measure = "[FULL_LIST_PM]")
                    
            IC_tree.write(IC_INDIRECT_GROUPWISE_xml_template) 
            
            # with [FULL_LIST_PM] of all non-IC based pairwise measures
            IC_tree = ET.parse(INDIRECT_GROUPWISE_xml_template)                                                                
            IC_root = IC_tree.getroot()
            IC_element_target = IC_root.find("./sml/measures/[@type='groupwise']")
            IC_new_measure = ET.SubElement(IC_element_target, 'measure', 
                                            id = pairwise_IC_item[2], 
                                            flag = pairwise_IC_item[1],
                                            pairwise_measure = "[FULL_LIST_PM]")
            
            IC_tree.write(INDIRECT_GROUPWISE_xml_template)             
       

            


# Note 

There are a few metrics available through SML that will not run in the same way as the others, as highlighted in the issues column of 'measures_revised'. 
They have been commented out of 'measures_revised', but the flags left in. 

You can manually create the XMLs for these using the templates and adjusting the file (unless the measure is invalid). 